In [1]:
# 02_main.ipynb
from grav_lens import get_datasets

from utils.model import create_model
from utils.loadsave import load_model_with_hyperparameters, load_hyperparameters, save_hyperparameters
from utils.optimize import dimensions, default_parameters


from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt.callbacks import CheckpointSaver
import time
import pickle

from tensorflow.keras import backend as K

from functools import partial

In [2]:
import os 

# mi carpeta data se encuentra en el root
home_data = os.path.join("..", "..")
# Ejemplo de uso
# INDEX 0 es una carpeta de datasets mucho mas chica
train_dataset, val_dataset, test_dataset = get_datasets(data_index='1', max_files=60, home=home_data)

for X, Y in train_dataset.take(1):  # Mostrar un batch de entrenamiento
    print("Train X:", X.shape)
    print("Train Y:", Y.shape)

for X, Y in val_dataset.take(1):  # Mostrar un batch de validación
    print("Val X:", X.shape)
    print("Val Y:", Y.shape)

for X, Y in test_dataset.take(1):  # Mostrar un batch de prueba
    print("Test X:", X.shape)
    print("Test Y:", Y.shape)


Using data folder: ..\..\data\1
Train X: (32, 128, 128, 3)
Train Y: (32, 128, 128, 1)
Val X: (12, 128, 128, 3)
Val Y: (12, 128, 128, 1)
Test X: (6, 128, 128, 3)
Test Y: (6, 128, 128, 1)


# Definiendo Funcion Optimizacion
Se ha intentado modularizar pero resulta en problems de pickling

In [3]:
best_loss = float('inf')
counter = 1
@use_named_args(dimensions=dimensions)
def F_objective(learning_rate, 
     h_kernel_size, hidden_filters, 
     out_kernel_size, 
     beta_1, beta_2, epsilon, 
     decay_steps, decay_rate, epochs):
    """
    Función objetivo para la optimización de hiperparámetros.
    """
    global best_loss, counter
    model = create_model(learning_rate, 
                        h_kernel_size, 
                        hidden_filters, out_kernel_size, 
                        beta_1, beta_2, 
                        epsilon,
                        decay_steps, decay_rate)

    model.fit(train_dataset, epochs=epochs, verbose=True)

    loss = model.evaluate(val_dataset, verbose=False)

    print(f"\nLoss: {loss:.2%}\n")

    if loss < best_loss:
        model.save_weights(f'best_model_{counter}.weights.h5')
        save_hyperparameters(
            learning_rate,
            h_kernel_size, hidden_filters, 
            out_kernel_size,
            beta_1, beta_2, epsilon, 
            decay_steps, 
            decay_rate, counter
        )
        print(f"Model weights and hyperparameters saved with ID: {counter}")
        counter += 1
        best_loss = loss

    K.clear_session()
    return loss

In [4]:
def run_hyp_optimize(dimensions, default_parameters, n_calls=17):
    best_loss = float('inf')
    counter = 1
    checkpoint_saver = CheckpointSaver("checkpoint.pkl", compress=9)

    start_time = time.time()
    res = gp_minimize(
        func=F_objective,  # Pasar la función parcial
        dimensions=dimensions,
        acq_func='EI', 
        n_calls=n_calls,
        x0=default_parameters,
        callback=[checkpoint_saver]
    )
    end_time = time.time()

    execution_time_minutes = (end_time - start_time) / 60
    print(f"Execution time: {execution_time_minutes:.2f} minutes")
    return res


In [5]:
# test optimize, corre una unica vez la funcion F a optimizar
# hyp_optimizer.run_test_optimize(train_dataset, val_dataset, verbose_train=True, verbose_val=True)

# La Gran Minimizacion
La siguiente funcion correra aproximadamente `n_calls` * el tiempo que corrio el paso anterior, debes tener en cuenta si se entrenan epocas y otras que aumenten el tiempo de manera no lineal dependiendo de la accion del hiperparametro

In [6]:
res = run_hyp_optimize(dimensions, default_parameters, n_calls=17)

# Guardar el objeto res, para hacer estadistica despues
with open('optimization_results.pkl', 'wb') as f:
    pickle.dump(res, f)

Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 43s 8s/step - loss: 22182.1777
Epoch 2/10


C:\Users\fbien.DESKTOP-6FMEAR7\Documents\python\lib\contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 32s 8s/step - loss: 24294.4688
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 32s 8s/step - loss: 23295.4434
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 38s 12s/step - loss: 22420.9902
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step - loss: 22309.4043
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 33s 8s/step - loss: 21279.1289
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 32s 8s/step - loss: 20908.4414
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 34s 8s/step - loss: 20576.6484
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 32s 8s/step - loss: 19624.5430
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 31s 8s/step - loss: 19279.9180

Loss: 267583.59%

Model weights and hyperparameters saved with ID: 1

Epoch 1/16


# Graficos

In [ ]:
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.plots import plot_histogram, plot_objective_2D

plot_convergence(res)
print(res.space)
print(res.x)
print("Accuracy: ", res.fun)

from skopt.plots import plot_histogram, plot_objective_2D

plot_histogram(result=res,  dimension_identifier='activation')
plot_objective(result=res, dimensions=dimension_names)
plot_evaluations(result=res, dimensions=dimension_names)
plt.show()